In [ ]:
import logging

import ssl4rs.utils

logger = ssl4rs.utils.logging.setup_logging_for_analysis_script(logging.INFO)
data_root_path = ssl4rs.utils.config.get_data_root_dir()
data_path = data_root_path / "fmow" / "fmow-rgb" / ".deeplake"
logger.info(f"Will attempt to load dataset at: {data_path}")

In [ ]:
import ssl4rs.data.parsers.fmow

parser = ssl4rs.data.parsers.fmow.DeepLakeParser(data_path)
parser.summary()

In [ ]:
# the default visualizer from deeplake doesn't work too well with datasets of varying lengths...
# (let's display some stuff manually instead)

import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import PIL.ImageDraw

max_display_count = 5

display_instance_idx = np.random.randint(len(parser))
print(f"Will display the different images of instance#{display_instance_idx}...")
batch = parser[display_instance_idx]
instance_id = batch["instance/id"]  # unique name (str) for this particular instance
images = batch["images/rgb/jpg"]  # list of rgb images, already decoded by default
metadata = batch["images/rgb/metadata"]  # list of json metadata dicts of the instance
bboxes = batch["images/rgb/bbox"]  # list of LTRB, pixel-wise instance bounding boxes
label = batch["instance/label"]  # class label of the instance (e.g. "airport")
subset = batch["instance/subset"]  # subset label of the instance (e.g. "train")
print(f"Instance: {instance_id} (class={label}, subset={subset})")
image_count = len(images)
if image_count > max_display_count:
    print(f"\tNumber of images: {image_count} (will display first {max_display_count})")
else:
    print(f"\tNumber of images: {image_count}")
display_count = min(max_display_count, image_count)

fig, axes = plt.subplots(nrows=1, ncols=display_count, squeeze=False, figsize=(5 * display_count, 5))
for img_idx in range(display_count):
    image = PIL.Image.fromarray(images[img_idx])
    drawer = PIL.ImageDraw.ImageDraw(image)
    bbox = bboxes[img_idx]
    left, top = bbox[0], bbox[1]
    right, bottom = left + bbox[2], top + bbox[3]
    drawer.rectangle((left, top, right, bottom), width=10, outline="red")
    axes[0][img_idx].imshow(image, aspect="auto")